This notebook contains a sequence of `numpy` and calculus exercises, with the goal of implementing the cross entropy loss function and its gradient.

Suppose we have $n$ data points and $k$ classes, and let ${\bf y}$ be the $k \times n$ matrix consisting of ground truths. The cross entropy loss for some predictions ${\bf p}$ is
$$E({\bf p}) = -{1 \over n}\sum_{i = 1}^n \sum_{j = 1}^k y_{ij} \ln p_{ij}.$$
Here is a non-vectorized implementation of cross entropy:

In [4]:
import numpy as np
import math

def CE_nonvectorized(y, p):
    """A nonvectorized implementation of cross entropy.

    Args:
        y: Ground truths, shape (k, n).
        p: Predictions, shape (k, n).

    Returns:
        (float) The cross entropy loss of p.
    """
    return -(1 / y.shape[1]) * sum(
        y[i, j] * math.log(p[i, j] + 1e-6) # add 1e-6 to allow for zeros and for stability
        for i in range(y.shape[0])
        for j in range(y.shape[1])
    )

### Exercise 1 (understanding check)
Given the toy ground truths `y` below, find a `p_close` with entires in `{0.1, 0.2, ..., 0.9}` that minimizes the loss of `p_close`:

In [3]:
y = np.array([
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1],
    [0, 1, 1, 0, 0]
])
p_init = np.array([
    [1/3, 1/3, 1/3, 1/3, 1/3],
    [1/3, 1/3, 1/3, 1/3, 1/3],
    [1/3, 1/3, 1/3, 1/3, 1/3]
])
p_close = np.array([
    [0.9, 0.0, 0.0, 0.1, 0.1],
    [0.0, 0.1, 0.1, 0.9, 0.9],
    [0.1, 0.9, 0.9, 0.0, 0.0]
])

print(f'loss of p_constant: {CE_nonvectorized(y, p_init)}')
print(f'loss of p_close: {CE_nonvectorized(y, p_close)}')

loss of p_constant: 1.0986092886726098
loss of p_close: 0.10535940454733242


### Exercise 2 (numpy)
Implement the following check that the columns of `p_init` and `p_close` sum to 1

In [7]:
def columns_sum_to_1(A):
    """
    Args:
        A: A two-dimensional matrix.

    Returns:
        (True/False) Whether the columns of the input matrix sum to 1.
    """
    return (np.sum(A, axis=0) == 1).all()

assert columns_sum_to_1(p_init)
assert columns_sum_to_1(p_close)

### Exercise 3 (calculus)
To vectorize the implementation of cross entropy, express $E({\bf p})$ using matrix operations, recalling the Hadamard product $\odot$ and denoting by $\text{sum}(A)$ the sum of the entires of a matrix $A$:
$$E({\bf p}) = -{1\over n}(y \odot \log(p))$$

### Exercise 4 (numpy)
Now write a vectorized implementation of cross entropy:

In [10]:
def CE_vectorized(y, p):
    """A vectorized implementation of cross entropy.

    Args:
        y: Ground truths, shape (k, n).
        p: Predictions, shape (k, n).

    Returns:
        (float) The cross entropy loss of p.
    """
    return -np.sum(y * np.log(p + 1e-6)) / y.shape[1]

assert CE_nonvectorized(y, p_init) == CE_vectorized(y, p_init)
assert CE_nonvectorized(y, p_close) == CE_vectorized(y, p_close)

Here is a speed comparison that shows that `CE_vectorized` is about 30 times faster than `CE_nonvectorized`:

In [13]:
from tqdm import tqdm

for _ in tqdm(range(20)):
    CE_vectorized(np.random.rand(10, 60000), np.random.rand(10, 60000))

for _ in tqdm(range(20)):
    CE_nonvectorized(np.random.rand(10, 60000), np.random.rand(10, 60000))

100%|██████████| 20/20 [00:10<00:00,  1.91it/s]


### Exercise 5 (calculus)
Now compute the gradient of $E({\bf p}),$ recalling the Hadamard division $\oslash$:
$$\nabla E({\bf p}) = -{1\over n}(y \oslash p)$$

### Exercise 6 (numpy)
Finally, write a vectorized implementation of the gradient of cross entropy:

In [20]:
def CE_gradient_vectorized(y, p):
    """A vectorized implementation of the gradient of cross entropy.

    Args:
        y: Ground truths, shape (k, n).
        p: Predictions, shape (k, n).

    Returns:
        The gradient of cross entropy, shape (k, n).
    """
    return -(y / (p + 1e-6)) / y.shape[1]

assert np.isclose(CE_gradient_vectorized(y, p_init), np.array([
    [-0.6,    0,    0,    0,    0],
    [   0,    0,    0, -0.6, -0.6],
    [   0, -0.6, -0.6,    0,    0],
])).all()

assert np.isclose(CE_gradient_vectorized(y, p_close), np.array([
    [-2/9,    0,    0,    0,    0],
    [   0,    0,    0, -2/9, -2/9],
    [   0, -2/9, -2/9,    0,    0],
])).all()

### Exercise 7 (understanding)
What is the relationship between the locations of the nonzero gradient entries and the ground-truths? Explain the following slogan: "cross entropy uses the carrot, whereas MSE uses the stick."

You can now copy-paste `CE_gradient_vectorized` and `CE_vectorized` (transpose the answer first) into `losses.py` for HW 3 Problem 2.